In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Define the directory containing the datasets
data_dir = "../../data/"
dataset_path = os.path.join(data_dir, "analyzed/catHarmQA/combined_catqa.csv")

In [3]:
data = pd.read_csv(dataset_path)

## 1. Original Question -> Response through all Models -> Labels from Llama Guard

In [13]:
# Count the number of safe and unsafe responses of each model for original questions
data.groupby("model")["original_response_safety"].value_counts(normalize=True).unstack()

original_response_safety,safe,unsafe
model,,
llama2,0.320000,0.680000
llama3,0.180000,0.820000
llama31,0.216364,0.783636
mistral,0.176364,0.823636


### 1.1 Cross verification

In [ ]:
# Define a helper function to filter the safety responses
def filter_safety_response(label):
    return label.strip().split()[0].lower()

# Load individual model safety data
llama2_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_llama2_Question_safety.csv"))
llama3_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_llama3_Question_safety.csv"))
llama31_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_llama31_Question_safety.csv"))
mistral_org_ques_safety_df = pd.read_csv(os.path.join(data_dir, "safety/catHarmQA/response/catqa_mistral_Question_safety.csv"))

# Apply filtering and calculate normalized proportions for each model
pd.DataFrame({
            "llama2": llama2_org_ques_safety_df['Question_llama2_safety'].apply(filter_safety_response).value_counts(normalize=True),
            "llama3": llama3_org_ques_safety_df['Question_llama3_safety'].apply(filter_safety_response).value_counts(normalize=True),
            "llama31": llama31_org_ques_safety_df['Question_llama31_safety'].apply(filter_safety_response).value_counts(normalize=True),
            "mistral": mistral_org_ques_safety_df['Question_mistral_safety'].apply(filter_safety_response).value_counts(normalize=True)
        }).fillna(0).T

,unsafe,safe
llama2,0.680000,0.320000
llama3,0.820000,0.180000
llama31,0.783636,0.216364
mistral,0.823636,0.176364


## 2. Original Question -> Get Unsafe Original Question using LLama-Guard -> Response through all Models -> Labels from Llama Guard

In [36]:
data.original_question_safety.value_counts(normalize=True)

original_question_safety
unsafe    0.932727
safe      0.067273
Name: proportion, dtype: float64

In [47]:
# filtered unsafe original question according to llama guard
data[data["original_question_safety"] == "unsafe"].groupby("model")["original_response_safety"].value_counts(normalize=True).unstack()

original_response_safety,safe,unsafe
model,,
llama2,0.280702,0.719298
llama3,0.130604,0.869396
llama31,0.169591,0.830409
mistral,0.126706,0.873294


## 3. Original Question -> Do Perturbation(char/word/sntnc) -> Check Safety of Perturbed Question -> Take only unsafe perturbed question -> Response through all Models -> Labels from Llama Guard

### 3.1. Safe percetnage of char, word, sentnc

In [74]:
# Safety percentages for perturbed questions
data[data["perturbed_question_safety"] == "unsafe"].groupby("perturbation_level").perturbed_response_safety.value_counts(normalize=True).unstack()

perturbed_response_safety,safe,unsafe
perturbation_level,,
char,0.181836,0.818164
sntnc,0.223295,0.776705
word,0.184400,0.815600


### 3.2. re-verification

In [91]:
for level in ["char","word", "sntnc"]:
    dd = data[data.perturbation_level == level]
    dd = dd[dd.perturbed_question_safety == "unsafe"]
    dd = dd.perturbed_response_safety.value_counts(normalize=True)*100
    print(f"{level} : {dd.safe:.2f}")

char : 18.18
word : 18.44
sntnc : 22.33


### 3.3. Unsafe

In [69]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(["model" , "perturbation_level"])["perturbed_response_safety"].value_counts(normalize=True).unstack() * 100

perturbed_response_safety        safe     unsafe
model   perturbation_level                      
llama2  char                28.177702  71.822298
        sntnc               35.113636  64.886364
        word                29.150157  70.849843
llama3  char                12.448323  87.551677
        sntnc               18.295455  81.704545
        word                12.973084  87.026916
llama31 char                16.444649  83.555351
        sntnc               19.659091  80.340909
        word                16.074600  83.925400
mistral char                15.663757  84.336243
        sntnc               16.250000  83.750000
        word                15.562235  84.437765

In [99]:
# Set display options
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Do not wrap columns
pd.set_option("display.max_colwidth", None)  # Do not truncate column values

data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["model", "perturbation_level", "category"]
)["perturbed_response_safety"].value_counts(normalize=True).unstack() * 100

In [104]:
data[data["perturbed_question_safety"] == "unsafe"].groupby(
    ["perturbation_level" , "category"]
)["perturbed_response_safety"].value_counts(normalize=True).unstack().T * 100

perturbation_level                 char                            \
category                  Adult Content Child Abuse Economic Harm   
perturbed_response_safety                                           
safe                          14.651708   12.629578      17.00977   
unsafe                        85.348292   87.370422      82.99023   

perturbation_level                                              \
category                  Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                        
safe                            15.739461            13.592896   
unsafe                          84.260539            86.407104   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_response_safety                                                  
safe                             11.601643       15.472103     27.350721   
unsafe                           88.398357       84.527897     72.649279   

perturbation_level                                                          \
category                  Political Campaigning Privacy Violation Activity   
perturbed_response_safety                                                    
safe                                  22.565543                  21.054688   
unsafe                                77.434457                  78.945312   

perturbation_level                                          sntnc              \
category                  Tailored Financial Advice Adult Content Child Abuse   
perturbed_response_safety                                                       
safe                                      30.230708     20.618557   13.372093   
unsafe                                    69.769292     79.381443   86.627907   

perturbation_level                                                            \
category                  Economic Harm Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                                      
safe                          23.148148       22.527473             21.91358   
unsafe                        76.851852       77.472527             78.08642   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_response_safety                                                  
safe                             14.516129       14.015152     30.813953   
unsafe                           85.483871       85.984848     69.186047   

perturbation_level                                                          \
category                  Political Campaigning Privacy Violation Activity   
perturbed_response_safety                                                    
safe                                  28.873239                   30.46875   
unsafe                                71.126761                   69.53125   

perturbation_level                                           word              \
category                  Tailored Financial Advice Adult Content Child Abuse   
perturbed_response_safety                                                       
safe                                      29.296875     15.180395   14.652677   
unsafe                                    70.703125     84.819605   85.347323   

perturbation_level                                                            \
category                  Economic Harm Fraud/Deception Hate/Harass/Violence   
perturbed_response_safety                                                      
safe                          17.398524       15.647226            14.298064   
unsafe                        82.601476       84.352774            85.701936   

perturbation_level                                                        \
category                  Illegal Activity Malware Viruses Physical Harm   
perturbed_res

In [105]:
data.category.value_counts(normalize=True) * 100

category
Adult Content                 9.090909
Child Abuse                   9.090909
Economic Harm                 9.090909
Fraud/Deception               9.090909
Hate/Harass/Violence          9.090909
Illegal Activity              9.090909
Malware Viruses               9.090909
Physical Harm                 9.090909
Political Campaigning         9.090909
Privacy Violation Activity    9.090909
Tailored Financial Advice     9.090909
Name: proportion, dtype: float64